# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [30]:
## imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [31]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [32]:
# Import the necessary CSVs to Pandas DataFrames

file_path = Path("Output_Data/housing_units.csv")
housing_units = pd.read_csv(file_path, index_col="year")


file_path = Path("Output_Data/sale_price.csv")
sale_price = pd.read_csv(file_path)

file_path = Path("Output_Data/gross_rent.csv")
gross_rent = pd.read_csv(file_path)

file_path = Path("Output_Data/avg_sale_price.csv")
avg_sale_neigh = pd.read_csv(file_path)

file_path = Path("Output_Data/top10_expensive.csv")
top10_most_expensive = pd.read_csv(file_path)

file_path = Path("Output_Data/most_expensive.csv")
most_expensive = pd.read_csv(file_path)

file_path = Path("Output_Data/neighbor_map.csv")
neighbor_map = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [33]:
# Define Panel Visualization Functions
def housing_units_per_year(housing_units):
    """Housing Units Per Year."""
    
    
    housing_units = housing_units.hvplot(
        title = "Number of Housing Units per Year",
        xlabel="Years", 
        ylabel="Housing Units",
        yformatter='%.0f'
    )
    return housing_units


def average_gross_rent(gross_rent):
    """Average Gross Rent in San Francisco Per Year."""
    
    gross_rent = gross_rent.hvplot(
        title = "Average Gross Rent in San Francisco Per Year",
        xlabel = "Years",
        ylabel = "Average Gross Rent",
        x="year"
       
    )
    return gross_rent



def average_sales_price(sale_price):
    """Average Sales Price Per Year."""
    
    sale_price= sale_price.hvplot(
        title = "Average Sales Price Per Year",
        xlabel = "Years",
        ylabel = "Average Sales Price",
        x = "year"
    )
    return sale_price



def average_price_by_neighborhood(avg_sale_neigh):
    """Average Prices by Neighborhood."""
    
    avg_sale_price = avg_sale_neigh.hvplot.line(
        groupby="neighborhood", 
        label=" ",
        xlabel="Year", 
        ylabel="Average Sale Price per Square Foot",
        x="year"
    )
    return avg_sale_price



def top_most_expensive_neighborhoods(top10_most_expensive):
    """Top 10 Most Expensive Neighborhoods."""

    top10_most_expensive = top10_most_expensive[["neighborhood","sale_price_sqr_foot"]].hvplot.bar(
        title = "Top 10 Expensive Neighborhoods in SFO",
        x="neighborhood",
        xlabel="Neighborhood", 
        ylabel ="Avg Sale Price per Square Foot", 
        rot=90,
        height=500
    )
    return top10_most_expensive


def most_expensive_neighborhoods_rent_sales(most_expensive):
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    most_expensive = most_expensive.hvplot.bar(
        groupby="neighborhood", 
        x ="year", 
        y = ['gross_rent','sale_price_sqr_foot'],
        xlabel = "Year",
        ylabel = "Gross Rent and Sale",
        rot = 90)

    return most_expensive
    
def parallel_coordinates(top10_most_expensive):
    """Parallel Coordinates Plot."""

    top10_most_expensive = top10_most_expensive.drop(columns=["Unnamed: 0"])
    top10_most_expensive = px.parallel_coordinates(
        top10_most_expensive,
        color = "sale_price_sqr_foot"
    )
    return top10_most_expensive



def parallel_categories(top10_most_expensive):
    """Parallel Categories Plot."""
    
    top10_most_expensive = px.parallel_categories(
        top10_most_expensive,
        color = "sale_price_sqr_foot"
    )
    return top10_most_expensive



def neighborhood_map(neighbor_map):
    """Neighborhood Map."""

    neighbor_map = px.scatter_mapbox(
        neighbor_map,
        lat = "Lat",
        lon = "Lon",
        size = "sale_price_sqr_foot",
        color = "gross_rent",
        title ="Average Sales Price per Square Foot and Gross Rent In San Francisco",
        zoom = 10,
        hover_name = "Neighborhood"
    )
    return neighbor_map

def sunburst(most_expensive):
    """Sunburst Plot."""
    most_expensive = px.sunburst(
        most_expensive,
        names = 'year',
        parents = 'neighborhood',
        values ='gross_rent',
        
    )
    return most_expensive
    
    
    


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [34]:
# Create a Title for the Dashboard
column = pn.Column(
    '# San Francisco Housing Analysis')


# Create a tab layout for the dashboard
row1 = pn.Row(housing_units_per_year(housing_units),average_gross_rent(gross_rent),average_sales_price(sale_price))
row2 = pn.Column(top_most_expensive_neighborhoods(top10_most_expensive),most_expensive_neighborhoods_rent_sales(most_expensive))
row3 = pn.Column(parallel_coordinates(top10_most_expensive),parallel_categories(top10_most_expensive))

analysis_dashboard = pn.Tabs(
    ("Welcome", neighborhood_map(neighbor_map)),
    ("Yearly Analysis", row1),
    ("Neighborhood Analysis", row2),
    ("Parallel Analysis", row3),
    ("Sunburst Chart", sunburst(most_expensive))
)


## Serve the Panel Dashboard

In [35]:
# Serve the# dashboard
analysis_dashboard.servable()

Tabs
    [0] Plotly(Figure, name='Welcome')
    [1] Row
        [0] HoloViews(Curve)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
    [2] Column
        [0] HoloViews(Bars)
        [1] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
    [3] Column
        [0] Plotly(Figure)
        [1] Plotly(Figure)
    [4] Plotly(Figure, name='Sunburst Chart')

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions. However, you can test each plot by uncommenting the following code

In [7]:
# housing_units_per_year(housing_units)

In [8]:
# average_gross_rent(gross_rent)

In [9]:
# average_sales_price(sale_price)

In [10]:
# average_price_by_neighborhood(avg_sale_neigh)

In [11]:
# top_most_expensive_neighborhoods(top10_most_expensive)

In [12]:
# most_expensive_neighborhoods_rent_sales(most_expensive)

In [13]:
# neighborhood_map(neighbor_map).show()

In [14]:
##  parallel_categories(top10_most_expensive)

In [15]:
# parallel_coordinates(top10_most_expensive)

In [16]:
## sunburst(most_expensive).show()